Variant 2
In this variant, you will compare two existing implementations of classifiers. You can choose any two
existing implementations of classification models. Train and test them on the dataset provided in the
beginning. Compare the two models using techniques for classification model comparison.

Reporting
Your submission for this assignment is a single PDF file with a report on the assignment. Your report
should be no longer than two pages. Somewhere at the top of the first page should be: your matric
number, full name, and a line “IN6227-2023-Assignment-1.2”. The only requirement for report
formatting is that it is readable, otherwise you are free to arrange information in any way you prefer.
Make sure to provide full performance comparison for the two models including the time it took to
train and apply the model. Explain all decisions you make along the way, e.g., how you fine-tune
model hyper-parameters, how you work with missing values, what is the stopping criterion, etc. If
you do any data pre-processing, please explain what and why was done.
Please upload your source code to GitHub and provide the repository link in the report.
Submission
Submission should be done in NTULearn. Access the assignment submission page through the left
navigation bar by selecting “Assignments”. Submit a single PDF file. Submissions are accepted up to
Friday, 3
rd March 2023, 23:59:59

In [1]:
import pandas as pd
#for random forest
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
#for MLP
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow import keras
#Preprocessing libraries
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

#https://archive.ics.uci.edu/ml/datasets/Census%2BIncome
headers = [
    'age','workclass', 'fnlwgt', 'education', 'edu-num', 'martial-status', 'occupation',
    'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country' , 'label'     
          ]
data =  pd.read_csv('adult.data', sep=",", names = headers) 
data.columns

#get test data
test =  pd.read_csv('adult.test', sep=",", names = headers)
test = test.iloc[1: , :] #get rid of first row

In [56]:
data.label.unique() #two kinds of tables only

array([' <=50K', ' >50K'], dtype=object)

In [2]:
def preprocessing(data,pca_num): #data is encoded, scaled and conducted PCA
    #encode the data
    encoder = LabelEncoder()
    for col in data.columns:
        data[col] = encoder.fit_transform(data[col])
    #split data set into features and label
    labels = data['label']
    features = data.drop("label", axis = 'columns')
    features = features
    #scale the data
    features = StandardScaler().fit_transform(features)
    #apply PCA
    length_of_PCA = pca_num
    PCA_columns = []
#     for num in range(1, length_of_PCA + 1):
#         PCA_columns.append('principal component ' + str(num)) 
#     pca = PCA(n_components= length_of_PCA)
    features = pca.fit_transform(features)
    return features,labels

In [3]:
#decision: to use Random Forest and MLP
#MLP
def preprocessing(data,pca_num): #data is encoded, scaled and conducted PCA
    #encode the data
    encoder = LabelEncoder()
    for col in data.columns:
        data[col] = encoder.fit_transform(data[col])
    #split data set into features and label
    labels = data['label']
    features = data.drop("label", axis = 'columns')
    features = features
    #scale the data
    features = StandardScaler().fit_transform(features)
    #apply PCA
#     length_of_PCA = pca_num
#     PCA_columns = []
#     for num in range(1, length_of_PCA + 1):
#         PCA_columns.append('principal component ' + str(num)) 
#     pca = PCA(n_components= length_of_PCA)
#     features = pca.fit_transform(features)
    return features,labels

In [16]:
#Using MLP with some basic tuning
component_num = 14
features, labels = preprocessing(data,component_num)
test_features,test_labels = preprocessing(test,component_num)
MLP = Sequential()
MLP.add(Dense(units = 50, input_shape=(14,), activation='relu'))
MLP.add(Dense(units = 200, activation='relu'))
MLP.add(Dense(1, activation='sigmoid'))
MLP.compile(optimizer='adam',
                loss= 'binary_crossentropy',
                metrics=['accuracy'])
MLP.fit(features,labels, epochs = 10,batch_size = 5)
MLP.evaluate(test_features,test_labels)

Epoch 1/10
6513/6513 [==============================] - 26s 4ms/step - loss: 0.3403 - accuracy: 0.8409
Epoch 2/10
6513/6513 [==============================] - 21s 3ms/step - loss: 0.3210 - accuracy: 0.8481
Epoch 3/10
6513/6513 [==============================] - 21s 3ms/step - loss: 0.3167 - accuracy: 0.8507
Epoch 4/10
6513/6513 [==============================] - 21s 3ms/step - loss: 0.3142 - accuracy: 0.8525
Epoch 5/10
6513/6513 [==============================] - 19s 3ms/step - loss: 0.3113 - accuracy: 0.8541
Epoch 6/10
6513/6513 [==============================] - 14s 2ms/step - loss: 0.3088 - accuracy: 0.8539
Epoch 7/10
6513/6513 [==============================] - 13s 2ms/step - loss: 0.3073 - accuracy: 0.8553
Epoch 8/10
6513/6513 [==============================] - 13s 2ms/step - loss: 0.3059 - accuracy: 0.8548
Epoch 9/10
6513/6513 [==============================] - 13s 2ms/step - loss: 0.3030 - accuracy: 0.8561
Epoch 10/10
509/509 [==============================] - 1s 1ms/step - loss

[0.3188125193119049, 0.8500092029571533]

In [12]:
#https://www.tensorflow.org/tutorials/keras/keras_tuner in order to test up some kind of tuning
import keras_tuner as kt
component_num = 14
features, labels = preprocessing(data,component_num)
test_features,test_labels = preprocessing(test,component_num)
def model_builder(hp):
    MLP = Sequential()
    hp_first_layer = hp.Int('first_layer', min_value=16, max_value=512, step=32)
    MLP.add(Dense(units = hp_first_layer, input_shape=(14,), activation='relu'))
    hp_second_layer = hp.Int('second_layer', min_value=16, max_value=512, step=32)
    MLP.add(Dense(units = hp_second_layer, activation='relu'))
    MLP.add(Dense(1, activation='sigmoid'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    MLP.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss= 'binary_crossentropy',
                metrics=['accuracy'])

    return MLP

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=30,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt'
                    )
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

#tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])
tuner.search(features, labels, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('first_layer')}, the optimal number of units in the second densely-connected
layer is {best_hps.get('second_layer')}, and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 88 Complete [00h 00m 29s]
val_accuracy: 0.8542914390563965

Best val_accuracy So Far: 0.8549055457115173
Total elapsed time: 00h 28m 26s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 368, the optimal number of units in the second densely-connected
layer is 304, and the optimal learning rate for the optimizer
is 0.0001.



In [13]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(features, labels, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
814/814 [==============================] - 4s 4ms/step - loss: 0.3812 - accuracy: 0.8273 - val_loss: 0.3390 - val_accuracy: 0.8376
Epoch 2/50
814/814 [==============================] - 3s 3ms/step - loss: 0.3347 - accuracy: 0.8444 - val_loss: 0.3285 - val_accuracy: 0.8446
Epoch 3/50
814/814 [==============================] - 3s 3ms/step - loss: 0.3259 - accuracy: 0.8481 - val_loss: 0.3283 - val_accuracy: 0.8457
Epoch 4/50
814/814 [==============================] - 3s 3ms/step - loss: 0.3219 - accuracy: 0.8499 - val_loss: 0.3235 - val_accuracy: 0.8494
Epoch 5/50
814/814 [==============================] - 3s 3ms/step - loss: 0.3185 - accuracy: 0.8514 - val_loss: 0.3233 - val_accuracy: 0.8477
Epoch 6/50
814/814 [==============================] - 3s 3ms/step - loss: 0.3162 - accuracy: 0.8538 - val_loss: 0.3217 - val_accuracy: 0.8515
Epoch 7/50
814/814 [==============================] - 3s 3ms/step - loss: 0.3144 - accuracy: 0.8541 - val_loss: 0.3210 - val_accuracy: 0.8488
Epoch 

In [14]:
hypermodel = tuner.hypermodel.build(best_hps)
# Retrain the model
hypermodel.fit(features, labels, epochs=best_epoch, validation_split=0.2)
eval_result = hypermodel.evaluate(test_features,test_labels)
print("[test loss, test accuracy]:", eval_result)

Epoch 1/23
814/814 [==============================] - 3s 3ms/step - loss: 0.3860 - accuracy: 0.8227 - val_loss: 0.3392 - val_accuracy: 0.8391
Epoch 2/23
814/814 [==============================] - 3s 3ms/step - loss: 0.3348 - accuracy: 0.8439 - val_loss: 0.3265 - val_accuracy: 0.8457
Epoch 3/23
814/814 [==============================] - 3s 3ms/step - loss: 0.3259 - accuracy: 0.8490 - val_loss: 0.3242 - val_accuracy: 0.8437
Epoch 4/23
814/814 [==============================] - 3s 4ms/step - loss: 0.3218 - accuracy: 0.8492 - val_loss: 0.3227 - val_accuracy: 0.8485
Epoch 5/23
814/814 [==============================] - 3s 3ms/step - loss: 0.3181 - accuracy: 0.8527 - val_loss: 0.3227 - val_accuracy: 0.8483
Epoch 6/23
814/814 [==============================] - 3s 4ms/step - loss: 0.3161 - accuracy: 0.8523 - val_loss: 0.3214 - val_accuracy: 0.8495
Epoch 7/23
814/814 [==============================] - 3s 3ms/step - loss: 0.3140 - accuracy: 0.8542 - val_loss: 0.3198 - val_accuracy: 0.8481
Epoch 